## Generating Optimal Multi-Line Tripping Scenarios on an 118-BUS System

### ChatGPT prompt: 
> __[optimal_risk_118](https://github.com/ckwon06/C.Lee--2025-Summer-Internship-Materials/blob/master/optimal_risk_118.ipynb)__ file linked
> 
> please read this thoroughly, this is code you have generated in the past. I want to execute the same type of experiment. Except, in addition to the most optimal single line tripping, do so again for two-line tripping, as well as three-line tripping. generate the code and explain the process along the way

### ChatGPT Response


#### Goal Summary
- Re-use the existing single-line optimal tripping method.
- Extend to double-line tripping (worst-case pairs).
- Extend to triple-line tripping (worst-case triples).
- Evaluate maximum system damage for each.

---

#### Step-by-Step Strategy

1. **Load Grid and Baseline Power Flow**
   - Load the 118-bus system.
   - Run `pp.runpp()` for the baseline.
   - Store baseline results.

2. **Define Evaluation Metric**
   - Metrics like sum of line loading percent, max line loading, total power losses, voltage deviations.
   - Use these to quantify damage.

3. **Single Line Tripping**
   - Loop through each line.
   - Temporarily disconnect one line.
   - Run power flow, evaluate damage.
   - Track worst case.

4. **Double Line Tripping**
   - Loop through combinations of 2 lines.
   - Disconnect both lines, run power flow, evaluate damage.
   - Track worst case.

5. **Triple Line Tripping**
   - Loop through combinations of 3 lines.
   - Same evaluation logic.
   - Note: computationally expensive (O(n³)).

---

#### Optimization Notes
- Number of combinations grows rapidly:
  - Single: ~186
  - Double: ~17,000
  - Triple: ~1,000,000
- To improve performance:
  - Filter to high loading lines.
  - Skip if power flow fails.
  - Consider parallel processing.

---

#### Next Step
Below is a code block you can insert directly into your notebook to perform these analyses.


In [1]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm import tqdm  # Optional: install tqdm for progress bars

# Load base network
net_base = pn.case118()
pp.runpp(net_base)

results = []

# -----------------------------
# Evaluation Helper Function
# -----------------------------
def evaluate_line_trip(trip_lines):
    net = pn.case118()
    for line in trip_lines:
        net.line.at[line, "in_service"] = False
    try:
        pp.runpp(net)
        overloaded = sum(net.res_line.loading_percent > 100)
        undervoltage = sum(net.res_bus.vm_pu < 0.95)
        loss = net.res_line.pl_mw.sum()
        return {
            "tripped_lines": trip_lines,
            "n_overloaded_lines": overloaded,
            "n_low_voltage_buses": undervoltage,
            "total_line_losses_mw": loss,
            "status": "OK"
        }
    except pp.LoadflowNotConverged:
        return {
            "tripped_lines": trip_lines,
            "n_overloaded_lines": np.nan,
            "n_low_voltage_buses": np.nan,
            "total_line_losses_mw": np.nan,
            "status": "Did Not Converge"
        }

# -----------------------------
# Single-Line Tripping
# -----------------------------
print("🟡 Running single-line tripping...")
for i in tqdm(range(len(net_base.line))):
    res = evaluate_line_trip([i])
    res["trip_type"] = "Single"
    results.append(res)

# -----------------------------
# Double-Line Tripping
# -----------------------------
print("🟠 Running double-line tripping...")
for i, j in tqdm(combinations(range(len(net_base.line)), 2), total=(len(net_base.line)*(len(net_base.line)-1))//2):
    res = evaluate_line_trip([i, j])
    res["trip_type"] = "Double"
    results.append(res)

# -----------------------------
# Triple-Line Tripping (filtered set)
# -----------------------------
print("🔴 Running triple-line tripping (filtered)...")

# Choose top 20 critical lines from previous failures or highest loading
critical_lines = pd.Series([r["tripped_lines"][0] for r in results if r["trip_type"]=="Single"])
top_lines = critical_lines.value_counts().head(20).index.tolist()

for i, j, k in tqdm(combinations(top_lines, 3)):
    res = evaluate_line_trip([i, j, k])
    res["trip_type"] = "Triple"
    results.append(res)

# -----------------------------
# DataFrame and Ranking
# -----------------------------
df_lines = pd.DataFrame(results)
df_lines["impact_score"] = (
    df_lines["n_overloaded_lines"].fillna(0)*5 +
    df_lines["n_low_voltage_buses"].fillna(0)*2 +
    df_lines["total_line_losses_mw"].fillna(0)*0.1
)

# Sort results
df_lines_sorted = df_lines.sort_values(by="impact_score", ascending=False).reset_index(drop=True)

# -----------------------------
# Final Output Table
# -----------------------------
print("📊 Top 10 Worst Line Tripping Scenarios")
display(df_lines_sorted.head(10))


🟡 Running single-line tripping...


100%|█████████████████████████████████████████| 173/173 [00:57<00:00,  3.00it/s]


🟠 Running double-line tripping...


100%|███████████████████████████████████| 14878/14878 [1:27:53<00:00,  2.82it/s]


🔴 Running triple-line tripping (filtered)...


1140it [06:27,  2.94it/s]

📊 Top 10 Worst Line Tripping Scenarios


,tripped_lines,n_overloaded_lines,n_low_voltage_buses,total_line_losses_mw,status,trip_type,impact_score
0,"[7, 89]",0.0,10.0,499.089218,OK,Double,69.908922
1,"[6, 89]",0.0,10.0,498.853629,OK,Double,69.885363
2,"[7, 33]",0.0,7.0,279.388720,OK,Double,41.938872
3,"[6, 33]",0.0,7.0,279.224025,OK,Double,41.922402
4,"[34, 89]",0.0,9.0,237.538798,OK,Double,41.753880
5,"[7, 34]",0.0,7.0,269.407703,OK,Double,40.940770
6,"[6, 34]",0.0,7.0,269.281043,OK,Double,40.928104
7,"[7, 27]",0.0,7.0,210.063907,OK,Double,35.006391
8,"[6, 27]",0.0,7.0,209.946297,OK,Double,34.994630
9,"[34, 66]",0.0,9.0,168.187082,OK,Double,34.818708


### ChatGPT Prompt

the method you are using may be a bit computationally expensive. Notice how you are checking every line/generator and gather information. Especially for two-line and three-line tripping, this could get seriously taxing. Instead, explore some different algorithms and methods to compute and get the same results and display the full code. Explain the process you used and go step by step.

### ChatGPT Response
---

#### ✅ Objective
Efficiently identify the most damaging line tripping scenarios (1, 2, and 3 lines) **without exhaustively checking every possibility**.

---

#### ⚙️ Optimized Method Overview

We use a tiered strategy:

1. **Prioritize by Initial Line Loading (Proxy for Criticality)**  
   Lines with the highest `loading_percent` are more likely to cause cascading issues if tripped.  
   We sort and select the top **N** most critical lines.

2. **Form Combinations Only Within This Critical Subset**  
   For example, with top 15 lines:  
   - Double-line combos = 105  
   - Triple-line combos = 455

3. **Evaluate Impact Using a Composite Metric**  
   Track:  
   - Number of overloaded lines  
   - Number of voltage violations  
   - MW losses  
   - (Optional) Slack bus overload or convergence failure

---

#### 📊 Algorithm Steps

#### Step 1: Load Network & Baseline Metrics

```python
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np
from itertools import combinations

# Load base system
net = pn.case118()
pp.runpp(net)
```

####  Step 2: Rank Lines by Base Loading

```python 
# Sort lines by descending loading
top_critical_lines = net.res_line.sort_values("loading_percent", ascending=False).head(15).index.tolist()
```

#### Step 3: Define Evaluation Function

```python 
def evaluate_outage(line_indices):
    test_net = pn.case118()
    for i in line_indices:
        test_net.line.in_service.at[i] = False
    try:
        pp.runpp(test_net)
        overloaded = sum(test_net.res_line.loading_percent > 100)
        undervoltage = sum(test_net.res_bus.vm_pu < 0.95)
        loss = test_net.res_line.pl_mw.sum()
        score = overloaded * 5 + undervoltage * 2 + loss * 0.1
        return {
            "lines_tripped": line_indices,
            "n_overloaded": overloaded,
            "n_undervoltage": undervoltage,
            "total_loss_mw": loss,
            "score": score,
            "status": "OK"
        }
    except pp.LoadflowNotConverged:
        return {
            "lines_tripped": line_indices,
            "n_overloaded": np.nan,
            "n_undervoltage": np.nan,
            "total_loss_mw": np.nan,
            "score": -1,
            "status": "DNC"
        }
```

#### Step 4: Run Efficient Experiments
```python 
results = []

# Single-line trip
for i in top_critical_lines:
    results.append(evaluate_outage([i]))

# Two-line trip
for i, j in combinations(top_critical_lines, 2):
    results.append(evaluate_outage([i, j]))

# Three-line trip
for i, j, k in combinations(top_critical_lines, 3):
    results.append(evaluate_outage([i, j, k]))

df = pd.DataFrame(results)
```

#### Step 5: Rank & Display
```python 
# Sort by damage score
df_sorted = df.sort_values(by="score", ascending=False).reset_index(drop=True)

# Show worst-case outages
print("Top 10 damaging line trip scenarios:")
print(df_sorted.head(10))

```

---

#### 🧠 Why This Is Efficient

**🔎 Base Filtering:**  
Instead of testing all possible combinations:

- ~17,000 double-line pairs  
- ~1,000,000 triple-line triplets  

We only test:

- **15** single-line outages  
- **105** two-line combinations  
- **455** three-line combinations  

**📌 Heuristic-Based Targeting:**  
Using `loading_percent` as a filtering metric is grounded in physical intuition:  
Lines with higher loading are more critical and more likely to:

- Be stressed under normal conditions  
- Trigger cascading failures if removed  
- Reveal key vulnerabilities in system design  


In [6]:
import pandapower as pp
import pandapower.networks as pn
import pandas as pd
import numpy as np
import copy
from itertools import combinations

# ---------------------------
# 1. Load Base Network
# ---------------------------
net = pn.case118()
pp.runpp(net)

# ---------------------------
# 2. Get Top-N Critical Lines
# ---------------------------
top_critical_lines = net.line.index.tolist()  # all lines


# ---------------------------
# 3. Define Outage Evaluator
# ---------------------------
def evaluate_outage(line_indices):
    net_test = copy.deepcopy(net)
    for i in line_indices:
        net_test.line.in_service.at[i] = False
    try:
        pp.runpp(net_test)
        overloaded = sum(net_test.res_line.loading_percent > 100)
        undervoltage = sum(net_test.res_bus.vm_pu < 0.95)
        loss = net_test.res_line.pl_mw.sum()
        score = overloaded * 5 + undervoltage * 2 + loss * 0.1
        return {
            "lines_tripped": line_indices,
            "n_overloaded": overloaded,
            "n_undervoltage": undervoltage,
            "total_loss_mw": loss,
            "score": score,
            "status": "OK"
        }
    except pp.LoadflowNotConverged:
        return {
            "lines_tripped": line_indices,
            "n_overloaded": np.nan,
            "n_undervoltage": np.nan,
            "total_loss_mw": np.nan,
            "score": -1,
            "status": "Did Not Converge"
        }

# ---------------------------
# 4. Run Tripping Scenarios
# ---------------------------
results = []

# Baseline Stress Score
baseline_loss = net.res_line.pl_mw.sum()
baseline_score = net.res_line.loading_percent.max() + np.abs(net.res_bus.vm_pu - 1.0).max()*100 + baseline_loss
print(f"Baseline Stress Score: {baseline_score:.2f}\n")

# SINGLE-LINE
print("Evaluating single-line trips...")
for i in top_critical_lines:
    results.append(evaluate_outage([i]))

# DOUBLE-LINE
print("Evaluating double-line trips...")
for i, j in combinations(top_critical_lines, 2):
    results.append(evaluate_outage([i, j]))

# TRIPLE-LINE
print("Evaluating triple-line trips...")
for i, j, k in combinations(top_critical_lines, 3):
    results.append(evaluate_outage([i, j, k]))

# ---------------------------
# 5. Create Result Table
# ---------------------------
df = pd.DataFrame(results)
df["lines_tripped_str"] = df["lines_tripped"].apply(lambda x: ", ".join(map(str, x)))
df["trip_count"] = df["lines_tripped"].apply(len)

# Compute impact score only where status is OK
df["impact_score"] = np.where(
    df["status"] == "OK",
    df["n_overloaded"] * 5 + df["n_undervoltage"] * 2 + df["total_loss_mw"] * 0.1,
    -1
)

df_sorted = df.sort_values(by="impact_score", ascending=False).reset_index(drop=True)

# ---------------------------
# 6. Display Final Rankings
# ---------------------------
print("\n✅ Computation Complete.")
print("🔧 Top 10 Most Damaging Outage Combinations:\n")
print(df_sorted[["lines_tripped_str", "trip_count", "impact_score", "n_overloaded", "n_undervoltage", "total_loss_mw", "status"]].head(10))

# Export sorted results with line tripping and impact scores
df_sorted.to_csv("real_risk_scores.csv", index=False)
print("✅ Exported risk scores to 'real_risk_scores.csv'")

Baseline Stress Score: 142.82

Evaluating single-line trips...
Evaluating double-line trips...
Evaluating triple-line trips...


KeyboardInterrupt: 

### ChatGPT Prompt
in addition to this graph, could you give me the top 10 most optimal trips for single, double and triple line trip respectively?

### ChatGPT Response

In [ ]:
# Format for display
def format_for_display(df):
    df_disp = df[[
        "lines_tripped_str",
        "trip_count",
        "impact_score",
        "n_overloaded",
        "n_undervoltage",
        "total_loss_mw",
        "status"
    ]].copy()
    df_disp.columns = [
        "lines", "n", "score", "overloads", "low_V", "loss_MW", "status"
    ]
    return df_disp.head(10)

# Filter by trip count
df_single = df_sorted[df_sorted.trip_count == 1]
df_double = df_sorted[df_sorted.trip_count == 2]
df_triple = df_sorted[df_sorted.trip_count == 3]

# Show top 10 from each
print("🔹 Top 10 Single-Line Trips:\n")
print(format_for_display(df_single).to_string(index=False))

print("\n🔸 Top 10 Double-Line Trips:\n")
print(format_for_display(df_double).to_string(index=False))

print("\n🔺 Top 10 Triple-Line Trips:\n")
print(format_for_display(df_triple).to_string(index=False))

# Export sorted results with line tripping and impact scores
# df_sorted.to_csv("real_risk_scores.csv", index=False)
# print("✅ Exported risk scores to 'real_risk_scores.csv'")
